In [9]:
!pip install -U -q PyDrive

In [10]:
from google.colab import files
uploaded = files.upload()

Saving lables.npy to lables (2).npy


In [11]:
!pip install tensorflow
!pip install keras

In [12]:
!pip install matplotlib
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.23.2)


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [14]:
lables = np.load('lables.npy')

In [15]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [16]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [17]:
link = 'https://drive.google.com/file/d/1eYtMUjfnc51NnmzLqj8fZJEbOIUxWenA/view?usp=sharing'

In [18]:
from google.colab import drive

In [19]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
data = np.load('/content/drive/My Drive/face_detection/data.npy')

In [21]:
data.shape

(3833, 224, 224, 3)

In [22]:
from sklearn.model_selection import train_test_split
(trainX, testX, trainY, testY) = train_test_split(data, lables,
	test_size=0.20, stratify=lables, random_state=42)

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator( featurewise_center=False, samplewise_center=False,
    featurewise_std_normalization=False, samplewise_std_normalization=False,
    zca_whitening=False, zca_epsilon=1e-06, rotation_range=15, width_shift_range=0.2,
    height_shift_range=0.2, brightness_range=None, shear_range=0.2, zoom_range=0.09,
    channel_shift_range=0.0, fill_mode='nearest', cval=0.0, horizontal_flip=False,
    vertical_flip=False, rescale=None, preprocessing_function=None,
    data_format=None, validation_split=0.0, dtype='float32'
)

In [ ]:
from tensorflow.keras.applications import MobileNetV2

base = MobileNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(224, 224, 3))
base.summary()

In [25]:
for layer in base.layers:
	layer.trainable = False

In [46]:

from tensorflow.keras.layers import GlobalAveragePooling2D,Flatten,Dense,Dropout,Input,AveragePooling2D

head = base.output
head = GlobalAveragePooling2D()(head)
head = Flatten(name="flatten")(head)
head = Dense(128, activation="relu")(head)
head = Dense(1, activation="sigmoid")(head)


In [47]:
from tensorflow.keras.models import Model
model = Model(inputs=base.input, outputs=head)


In [48]:
INIT_LR = 1e-4
EPOCHS = 1
BS = 32

In [49]:

from tensorflow.keras.optimizers import Adam
#opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer="Adam",
	metrics=["accuracy"])


In [50]:
history = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

95/95 [==============================] - 148s 2s/step - loss: 0.1236 - accuracy: 0.9532 - val_loss: 0.0282 - val_accuracy: 0.9870


In [51]:
predIdxs = model.predict(testX, batch_size=BS)

In [52]:
predIdxs = np.argmax(predIdxs, axis=1)

In [58]:


# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model.h5') 

In [59]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model  my_model.h5
assets	saved_model.pb	variables


In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Save Keras Model or weights on google drive

# create on Colab directory
model.save('model.h5')    
model_file = drive.CreateFile({'title' : 'model.h5'})
model_file.SetContentFile('model.h5')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1DU8cJCY3toysbz27yZaFMAqFuiwS9UPK'})